In [42]:
import os
import pickle
import mlflow
import numpy as np

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll import scope
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("random-forest-hyperopt")


<Experiment: artifact_location='/workspaces/Testrepo/mlruns/2', creation_time=1723539801480, experiment_id='2', last_update_time=1723539801480, lifecycle_stage='active', name='random-forest-hyperopt', tags={}>

In [64]:
def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)

def run_optimization(data_path: str, num_trials: int):

    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))

    def objective(params):

        with mlflow.start_run():
            mlflow.log_param("max_depth", params["max_depth"])
            mlflow.log_param("min_samples_leaf", params["min_samples_leaf"])
            mlflow.log_param("min_samples_split", params["min_samples_split"])
            mlflow.log_param("n_estimators", params["n_estimators"])
            mlflow.log_param("random_state", params["random_state"])
            
            rf = RandomForestRegressor(**params)
            rf.fit(X_train, y_train)
            y_pred = rf.predict(X_val)
            
            dementia_threshold = np.log1p(1.5/26)
            y_pred = (y_pred > dementia_threshold).astype(int)
            
            score = roc_auc_score(y_val,y_pred)          
            
            mlflow.log_metric("Roc_AUC_score", score)
            mlflow.sklearn.log_model(RandomForestRegressor, artifact_path="artifact")
            mlflow.set_tag("model", rf)
            mlflow.end_run()
        return {'loss': -score, 'status': STATUS_OK}

    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 1, 20, 1)),
        'n_estimators': scope.int(hp.quniform('n_estimators', 10, 50, 1)),
        'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
        'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 4, 1)),
        'random_state': 42
    }
    rstate = np.random.default_rng(42)  # for reproducible results
    fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=num_trials,
        trials=Trials(),
        rstate=rstate
    )


In [44]:
model = LogisticRegression(solver='liblinear', C=1, max_iter=1000, random_state=42, class_weight='balanced')

X_train, y_train = load_pickle(os.path.join("./output", "train.pkl"))
X_val, y_val = load_pickle(os.path.join("./output", "val.pkl"))



In [45]:
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
roc_auc_score(y_val,y_pred)

0.5669014084507042

In [67]:
run_optimization(data_path="./output", num_trials=2)

  0%|          | 0/2 [00:00<?, ?trial/s, best loss=?]

2024/08/15 09:45:01 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



 50%|█████     | 1/2 [00:01<00:01,  1.98s/trial, best loss: -0.7053990610328639]

2024/08/15 09:45:03 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



100%|██████████| 2/2 [00:03<00:00,  2.00s/trial, best loss: -0.7053990610328639]


In [48]:
#mlflow.end_run()